In [1]:
library(AER)
library(stargazer)

Loading required package: car
Loading required package: carData
Loading required package: lmtest
Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Loading required package: sandwich
Loading required package: survival

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



Z
  must satisfy two conditions to be a valid instrument:

1. Instrument relevance condition:

X  and its instrument Z must be correlated: 

$$\rho_{Z_i,X_i} \neq 0$$

2. Instrument exogeneity condition:

The instrument Z  must not be correlated with the error term 


$$\rho_{Z_i,u_i} = 0$$

In [2]:
# load the data set and get an overview
library(AER)
data("CigarettesSW")
summary(CigarettesSW)

     state      year         cpi          population           packs       
 AL     : 2   1985:48   Min.   :1.076   Min.   :  478447   Min.   : 49.27  
 AR     : 2   1995:48   1st Qu.:1.076   1st Qu.: 1622606   1st Qu.: 92.45  
 AZ     : 2             Median :1.300   Median : 3697472   Median :110.16  
 CA     : 2             Mean   :1.300   Mean   : 5168866   Mean   :109.18  
 CO     : 2             3rd Qu.:1.524   3rd Qu.: 5901500   3rd Qu.:123.52  
 CT     : 2             Max.   :1.524   Max.   :31493524   Max.   :197.99  
 (Other):84                                                                
     income               tax            price             taxs       
 Min.   :  6887097   Min.   :18.00   Min.   : 84.97   Min.   : 21.27  
 1st Qu.: 25520384   1st Qu.:31.00   1st Qu.:102.71   1st Qu.: 34.77  
 Median : 61661644   Median :37.00   Median :137.72   Median : 41.05  
 Mean   : 99878736   Mean   :42.68   Mean   :143.45   Mean   : 48.33  
 3rd Qu.:127313964   3rd Qu.:50.88   

The idea is that  
S
a
l
e
s
T
a
x
  is a relevant instrument as it is included in the after-tax average price per pack. Also, it is plausible that  
S
a
l
e
s
T
a
x
  is exogenous since the sales tax does not influence quantity sold directly but indirectly through the price.

In [3]:
# compute real per capita prices
CigarettesSW$rprice <- with(CigarettesSW, price / cpi)

#  compute the sales tax
CigarettesSW$salestax <- with(CigarettesSW, (taxs - tax) / cpi)

# check the correlation between sales tax and price
cor(CigarettesSW$salestax, CigarettesSW$price)

[1] 0.6141228

In [4]:
# generate a subset for the year 1995
c1995 <- subset(CigarettesSW, year == "1995")

In [5]:
# perform the first stage regression
cig_s1 <- lm(log(rprice) ~ salestax, data = c1995)

coeftest(cig_s1, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error  t value  Pr(>|t|)    
(Intercept) 4.6165463  0.0289177 159.6444 < 2.2e-16 ***
salestax    0.0307289  0.0048354   6.3549 8.489e-08 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [6]:
# inspect the R^2 of the first stage regression
summary(cig_s1)$r.squared

[1] 0.4709961

In [7]:
# store the predicted values
lcigp_pred <- cig_s1$fitted.values

In [8]:
# run the stage 2 regression
cig_s2 <- lm(log(c1995$packs) ~ lcigp_pred)
coeftest(cig_s2, vcov = vcovHC)


t test of coefficients:

            Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  9.71988    1.70304  5.7074 7.932e-07 ***
lcigp_pred  -1.08359    0.35563 -3.0469  0.003822 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [9]:
coeftest(cig_s2, vcov = vcovHC)


t test of coefficients:

            Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  9.71988    1.70304  5.7074 7.932e-07 ***
lcigp_pred  -1.08359    0.35563 -3.0469  0.003822 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


The function ivreg() from the package AER carries out TSLS procedure automatically. It is used similarly as lm(). Instruments can be added to the usual specification of the regression formula using a vertical bar separating the model equation from the instruments. Thus, for the regression at hand the correct formula is log(packs) ~ log(rprice) | salestax.

In [10]:
# perform TSLS using 'ivreg()'
cig_ivreg <- ivreg(log(packs) ~ log(rprice) | salestax, data = c1995)

coeftest(cig_ivreg, vcov = vcovHC, type = "HC1")


t test of coefficients:

            Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  9.71988    1.52832  6.3598 8.346e-08 ***
log(rprice) -1.08359    0.31892 -3.3977  0.001411 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


suggests that an increase in cigarette prices by one percent reduces cigarette consumption by roughly  
1.08
 percentage points, which is fairly elastic. However, we should keep in mind that this estimate might not be trustworthy even though we used IV estimation: there still might be a bias due to omitted variables. Thus a multiple IV regression approach is needed.

In [11]:
# add rincome to the dataset
CigarettesSW$rincome <- with(CigarettesSW, income / population / cpi)

c1995 <- subset(CigarettesSW, year == "1995")

In [12]:
# estimate the model
cig_ivreg2 <- ivreg(log(packs) ~ log(rprice) + log(rincome) | log(rincome) + 
                    salestax, data = c1995)

coeftest(cig_ivreg2, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   9.43066    1.25939  7.4883 1.935e-09 ***
log(rprice)  -1.14338    0.37230 -3.0711  0.003611 ** 
log(rincome)  0.21452    0.31175  0.6881  0.494917    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [13]:
# add cigtax to the data set
CigarettesSW$cigtax <- with(CigarettesSW, tax/cpi)

c1995 <- subset(CigarettesSW, year == "1995")

In [14]:
# estimate the model
cig_ivreg3 <- ivreg(log(packs) ~ log(rprice) + log(rincome) | 
                    log(rincome) + salestax + cigtax, data = c1995)

coeftest(cig_ivreg3, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   9.89496    0.95922 10.3157 1.947e-13 ***
log(rprice)  -1.27742    0.24961 -5.1177 6.211e-06 ***
log(rincome)  0.28040    0.25389  1.1044    0.2753    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [17]:
# estimate the model
cig_ivreg3 <- ivreg(log(packs) ~ log(rprice) + log(rincome) | 
                   .-log(rprice) + salestax + cigtax, data = c1995)

coeftest(cig_ivreg3, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   9.89496    0.95922 10.3157 1.947e-13 ***
log(rprice)  -1.27742    0.24961 -5.1177 6.211e-06 ***
log(rincome)  0.28040    0.25389  1.1044    0.2753    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [18]:
# subset data for year 1985
c1985 <- subset(CigarettesSW, year == "1985")

# define differences in variables
packsdiff <- log(c1995$packs) - log(c1985$packs)

pricediff <- log(c1995$price/c1995$cpi) - log(c1985$price/c1985$cpi)

incomediff <- log(c1995$income/c1995$population/c1995$cpi) -
log(c1985$income/c1985$population/c1985$cpi)

salestaxdiff <- (c1995$taxs - c1995$tax)/c1995$cpi - (c1985$taxs - c1985$tax)/c1985$cpi

cigtaxdiff <- c1995$tax/c1995$cpi - c1985$tax/c1985$cpi

In [19]:
# estimate the three models
cig_ivreg_diff1 <- ivreg(packsdiff ~ pricediff + incomediff | incomediff + 
                         salestaxdiff)

cig_ivreg_diff2 <- ivreg(packsdiff ~ pricediff + incomediff | incomediff + 
                         cigtaxdiff)

cig_ivreg_diff3 <- ivreg(packsdiff ~ pricediff + incomediff | incomediff + 
                         salestaxdiff + cigtaxdiff)

In [21]:
# robust coefficient summary for 1.
coeftest(cig_ivreg_diff1, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept) -0.117962   0.068217 -1.7292   0.09062 .  
pricediff   -0.938014   0.207502 -4.5205 4.454e-05 ***
incomediff   0.525970   0.339494  1.5493   0.12832    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [22]:
# robust coefficient summary for 2.
coeftest(cig_ivreg_diff2, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept) -0.017049   0.067217 -0.2536    0.8009    
pricediff   -1.342515   0.228661 -5.8712 4.848e-07 ***
incomediff   0.428146   0.298718  1.4333    0.1587    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [23]:
# robust coefficient summary for 3.
coeftest(cig_ivreg_diff3, vcov = vcovHC, type = "HC1")


t test of coefficients:

             Estimate Std. Error t value  Pr(>|t|)    
(Intercept) -0.052003   0.062488 -0.8322    0.4097    
pricediff   -1.202403   0.196943 -6.1053 2.178e-07 ***
incomediff   0.462030   0.309341  1.4936    0.1423    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [24]:
# gather robust standard errors in a list
rob_se <- list(sqrt(diag(vcovHC(cig_ivreg_diff1, type = "HC1"))),
               sqrt(diag(vcovHC(cig_ivreg_diff2, type = "HC1"))),
               sqrt(diag(vcovHC(cig_ivreg_diff3, type = "HC1"))))

# generate table
stargazer(cig_ivreg_diff1, cig_ivreg_diff2,cig_ivreg_diff3,
  header = FALSE, 
  type = "html",
  omit.table.layout = "n",
  digits = 3, 
  column.labels = c("IV: salestax", "IV: cigtax", "IVs: salestax, cigtax"),
  dep.var.labels.include = FALSE,
  dep.var.caption = "Dependent Variable: 1985-1995 Difference in Log per Pack Price",
  se = rob_se)


<table style="text-align:center"><tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="3">Dependent Variable: 1985-1995 Difference in Log per Pack Price</td></tr>
<tr><td></td><td colspan="3" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>IV: salestax</td><td>IV: cigtax</td><td>IVs: salestax, cigtax</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td></tr>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">pricediff</td><td>-0.938<sup>***</sup></td><td>-1.343<sup>***</sup></td><td>-1.202<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(0.208)</td><td>(0.229)</td><td>(0.197)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">incomediff</td><td>0.526</td><td>0.428</td><td>0.462</td></tr>
<tr><td style="text-align:left"></td

This hinges on the validity of the instruments used. To assess this we compute  
F
 -statistics for the first-stage regressions of all three models to check instrument relevance.

In [25]:
# first-stage regressions
mod_relevance1 <- lm(pricediff ~ salestaxdiff + incomediff)
mod_relevance2 <- lm(pricediff ~ cigtaxdiff + incomediff)
mod_relevance3 <- lm(pricediff ~ incomediff + salestaxdiff + cigtaxdiff)

In [26]:
# check instrument relevance for model (1)
linearHypothesis(mod_relevance1, 
                 "salestaxdiff = 0", 
                 vcov = vcovHC, type = "HC1")

Res.Df,Df,F,Pr(>F)
46,NA,NA,NA
45,1,28.4455,3.008616e-06


In [27]:
# check instrument relevance for model (2)
linearHypothesis(mod_relevance2, 
                 "cigtaxdiff = 0", 
                 vcov = vcovHC, type = "HC1")

Res.Df,Df,F,Pr(>F)
46,NA,NA,NA
45,1,98.03443,7.090439e-13


In [28]:
# check instrument relevance for model (3)
linearHypothesis(mod_relevance3, 
                 c("salestaxdiff = 0", "cigtaxdiff = 0"), 
                 vcov = vcovHC, type = "HC1")

Res.Df,Df,F,Pr(>F)
46,NA,NA,NA
44,2,76.91646,4.339013e-15


also conduct the overidentifying restrictions test for model three which is the only model where the coefficient on the difference in log prices is overidentified (  ) such that the  J -statistic can be computed. To do this we take the residuals stored in cig_ivreg_diff3 and regress them on both instruments and the presumably exogenous regressor incomediff. We again use linearHypothesis() to test whether the coefficients on both instruments are zero which is necessary for the exogeneity assumption to be fulfilled. Note that with test = “Chisq” we obtain a chi-squared distributed test statistic instead of an  
F -statistic.

In [29]:
# compute the J-statistic
cig_iv_OR <- lm(residuals(cig_ivreg_diff3) ~ incomediff + salestaxdiff + cigtaxdiff)

cig_OR_test <- linearHypothesis(cig_iv_OR, 
                               c("salestaxdiff = 0", "cigtaxdiff = 0"), 
                               test = "Chisq")
cig_OR_test

Res.Df,RSS,Df,Sum of Sq,Chisq,Pr(>Chisq)
46,0.3747215,NA,NA,NA,NA
44,0.3369523,2,0.03776915,4.931982,0.08492463


Caution: In this case the  p -Value reported by linearHypothesis() is wrong because the degrees of freedom are set to  
2 . This differs from the degree of overidentification (  ) so the  J -statistic is  χ21  distributed instead of following a  
χ22  distribution as assumed defaultly by linearHypothesis(). We may compute the correct  p -Value using pchisq().

In [30]:
# compute correct p-value for J-statistic
pchisq(cig_OR_test[2, 5], df = 1, lower.tail = FALSE)

[1] 0.02636406

Since this value is smaller than  
0.05
  we reject the hypothesis that both instruments are exogenous at the level of  
5
%
 . This means one of the following:

The sales tax is an invalid instrument for the per-pack price.
The cigarettes-specific sales tax is an invalid instrument for the per-pack price.
Both instruments are invalid.
The book argues that the assumption of instrument exogeneity is more likely to hold for the general sales tax (see Chapter 12.4 of the book) such that the IV estimate of the long-run elasticity of demand for cigarettes we consider the most trustworthy is  
−
0.94
 , the TSLS estimate obtained using the general sales tax as the only instrument.

The interpretation of this estimate is that over a 10-year period, an increase in the average price per package by one percent is expected to decrease consumption by about  
0.94
  percentage points. This suggests that, in the long run, price increases can reduce cigarette consumption considerably.